Description:

This project is a variation of the card game Durak, played between one human player and an artificial intelligence player.

In this version, both players are dealt 6 cards initially, and one additional card is drawn from that deck, who's suit represents the Trump Suit. The player starts by placing down a card, and the AI must place a card down that beats the previous card. 

Conditions for beating the current card are as follows:

1.) If the current card is not a trump card, it can be beaten by a card of the same suit and a higher rank, OR by any trump card.
2.) If the current card is a trump card, it can only be beaten by a higher trump card.

After the AI does this, the player must do the same, and both continue to do so until one player cannot or chooses not to beat the previous card. When this happens, the player who did not beat the previous card will take all of the cards that have been placed into the middle into their hands (except for trump cards which can only be used once in order to prevent a stalemate). 

The first player to get rid of all of their cards wins.

In [0]:
# This code cell has the class and function declarations. It is the same
# code that is included in the modules section of the directory.

import random # Looked up on docs.python.org

class Card:
  """
  A Card object-> just has a rank and a suit
  """
  def __init__(self, rank = None, suit = None):
      self.rank = rank
      self.suit = suit
    
  def __repr__(self):
    """
    Allows the card to be printed out as "[Rank] of [Suit]"
    """
    return str(self.rank) + " of " + self.suit 

class Deck:
  """
  A Deck object-> represents a deck of cards. For durak, a 36-card deck is used
  in which cards 2 through 5 are excluded
  """
  def __init__(self):
      self.suits = ["Diamonds", "Clubs", "Hearts", "Spades"]
      self.ranks = ["Ace", 6 , 7, 8, 9, 10, "Jack", "King", "Queen"]
      self.cards = []
      self._initialize_cards()

  def _initialize_cards(self):
    """
    Helper function for the constructor, just used to populate deck.cards
    """
    for suit in self.suits: # For each suit
        for rank in self.ranks: # Add one card per rank
            self.cards.append(Card(rank, suit))

  def shuffle(self):
    """
    Shuffles the deck
    """
    random.shuffle(self.cards)

class Game:
  """
  Represents all cards in play during a game.
  """
  def __init__(self):
      self.num_players = 2 # One human player, one AI
      self.deck = Deck() # Contains a deck
      self.hands = [[] for i in range(self.num_players)] # 1 hand per player
      self.deck.shuffle()
    
  def deal(self, num_cards = 6):
    """
    Deals 6 cards to each player to start off -> in Durak, each player is
    supposed to begin with 6 cards.
    """
    for player in range(self.num_players):
        for card in range(num_cards):
            self.hands[player].append(self.deck.cards[-1])
            self.deck.cards.pop() # Looked up list.pop() on Stack Overflow
  
def getMinRank(hand):
  """ 
  Gets the card with the minimum rank of a list of cards
  """
  ranks = {'Ace': 14, 6:6, 7:7, 8:8, 9:9, 10:10, 'Jack': 11, 'King':13,
                 'Queen':12}
  minRank = min(ranks[card.rank] for card in hand) # min() from docs.python.org
  minCard = None
  for card in hand:
    if ranks[card.rank] == minRank:
      minCard = card
  return minCard

In [0]:
# This code cell contains the main game implementation. It uses the classes
# and function created in the code cell above. In order to play the game,
# run the code cell below this one (after you run this one and the previous,
# of course).


def durak():
  """
  Main Game
  """

  # The following block initializes the beginning state of the game:
  # 1.) Deals cards
  # 2.) Selects Trump Card
  # 3.) No current card is in middle yet, so cards on the table is also empty
  my_durak = Game()
  my_durak.deal()
  trump_suit = my_durak.deck.cards.pop().suit 
  print("Trump Suit:", trump_suit, "\n")
  ranks = {'Ace': 14, 6:6, 7:7, 8:8, 9:9, 10:10, 'Jack': 11, 'King':13,
                 'Queen':12}
  turn = 0 # 0 represents player turn, 1 represents AI turn
  current_card = None 
  cards_on_table = []
  game_running = True
  check = False # When one player has gotten rid of all their cards

  # The following is the main algorithm of the game
  while game_running == True:
    if current_card == None:
      print("No cards in middle yet: place first card\n")

    if turn == 0: # If is player's turn

      # The following block creates a list of cards that would beat the 
      # current card in play.
      # 1.) If no card has been played yet, any card in the hand will work.
      # 2.) If the card in the middle is not a trump, it can be beaten by
      #     a card of the same suit with a higher rank or by any trump.
      # 3.) If the card in the middle is a trump, it can only be beaten by 
      #     another trump of a higher rank.
      valid_choices = []
      for card in my_durak.hands[0]: 
        current_trump = (current_card != None and 
                         current_card.suit == trump_suit)
        outrank = (current_card != None and 
                   ranks[card.rank] > ranks[current_card.rank] 
                   and card.suit == current_card.suit)
        if (current_card == None or (current_trump and outrank) or 
            ((not current_trump) and (card.suit == trump_suit or outrank))):
          valid_choices.append(card)

      print("\nThis is your current hand: ", my_durak.hands[0])

      if len(valid_choices) > 0: # If some cards would win
        print('The following cards would beat the current card:')

        for index in range(len(valid_choices)):
          print("Enter", index, "to select", valid_choices[index])
        print("OR enter -1 to PASS and Collect Middle")

        selection_index = int(input("\n"))

        # The following while loop makes sure the player entered a valid card
        while selection_index > len(valid_choices)-1 or selection_index < -1:
          selection_index = int(input("Please enter a valid choice\n"))

        # If player chooses not to play a winning card
        if selection_index == -1:
          if check == True: # If AI has no more cards
            game_running = False

          input('\nYou PASSED-> PRESS ENTER to Collect Middle\n')

          # Collects middle, except for Trump cards which are discarded
          my_durak.hands[0].extend([card for card in cards_on_table 
                                   if card.suit != trump_suit])
          cards_on_table = []
          current_card = None
        
        # If the player picked a card to play
        else:
          if check == True: # If the AI has 0 cards
            check = False # Since the AI will have to collect the middle cards

          selected = valid_choices[selection_index]
          cards_on_table.append(selected)
          my_durak.hands[0].remove(selected)
          current_card = cards_on_table[-1]

          if len(my_durak.hands[0]) == 0: # If this card was player's last
            check = True # AI has to beat it next turn or game is over

      else: # If no cards would win, player collects middle cards
        if check == True:
          game_running = False

        input("\nNone of your cards can win-> PRESS ENTER to Collect Middle\n")
        my_durak.hands[0].extend([card for card in cards_on_table 
                                 if card.suit != trump_suit])
        cards_on_table = []
        current_card = None
      turn = 1

    elif turn == 1: #if AI turn

      trump_choices = [] # Trump cards the AI has that would beat current card
      non_trumps = [] # Non-trump cards that would beat the current card

      # The following determines which cards are valid options in the same way
      # that was done for the player, except this time it separates the choices
      # based on whether they are trump cards or not.
      for card in my_durak.hands[1]:
        current_trump = (current_card != None and 
                         current_card.suit == trump_suit)
        outrank = (current_card != None and 
                   ranks[card.rank] > ranks[current_card.rank] 
                   and card.suit == current_card.suit)
        if (current_card == None or (current_trump and outrank) or 
            ((not current_trump) and (card.suit == trump_suit or outrank))):
          if card.suit == trump_suit:
            trump_choices.append(card)
          else:
            non_trumps.append(card)

      # If no cards would win
      if len(trump_choices) == 0 and len(non_trumps) == 0:
        if check == True: # If player has no cards, game over
          game_running = False

        print('\nAI has no winning cards: it collects the cards in middle\n')
        my_durak.hands[1].extend([card for card in cards_on_table 
                                  if card.suit != trump_suit])
        cards_on_table = []
        current_card = None


      # If possible, the AI would prefer to play a non-trump instead of a Trump,
      # and it would prefer to play the card with the lowest rank that can beat
      # the current card
      elif len(non_trumps) > 0:  
        if check == True:
          check = False

        best_choice = getMinRank(non_trumps)
        print("\nThe AI has played:\n", best_choice)
        cards_on_table.append(best_choice)
        my_durak.hands[1].remove(best_choice)
        current_card = cards_on_table[-1]
      
        if len(my_durak.hands[1]) == 0:
          check = True

      # If only a Trump can win, the AI chooses to play the trump with the
      # lowest rank that can beat the current card
      else:
        if check == True:
          check = False

        best_choice = getMinRank(trump_choices)
        print("The AI has played:\n", best_choice)
        cards_on_table.append(best_choice)
        my_durak.hands[1].remove(best_choice)
        current_card = cards_on_table[-1]
        
        if len(my_durak.hands[1]) == 0:
          check = True
      turn = 0

  # If player managed to get rid of all their cards
  if len(my_durak.hands[0]) == 0:
    print("The player wins. The AI is the durak")
  
  # If the AI managed to get rid of all their cards
  elif len(my_durak.hands[1]) == 0:
    print("The AI wins. The player is the durak")

In [0]:
# This runs the game
durak()